In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/StarGAN_128/'

In [4]:
pretrained = ''
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b1' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 200
start_epoch = 0
train_batch = 190
test_batch = 190
lr = 0.015
schedule = [20, 75, 125, 175]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/star/128/b1_2' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    
train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.3, 'drop_connect_rate':0.3})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 6.52M


In [10]:
# summary(model, input_size=(3,64,64), device='cuda')

# Loss

In [11]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4)
# optimizer = optim.Adam(model.parameters(), weight_decay=1e-4)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

# Train

In [13]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [14]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [15]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [1 | 200] LR: 0.015000
1/1445 Data:1.252 | Batch:8.088 | Total:0:00:08 | ETA:3:14:42 | Loss:0.6965400576591492 | top1:51.05263900756836
11/1445 Data:0.002 | Batch:0.665 | Total:0:00:14 | ETA:0:33:40 | Loss:0.7006459778005426 | top1:50.239234924316406
21/1445 Data:0.003 | Batch:0.661 | Total:0:00:21 | ETA:0:15:42 | Loss:0.7025354731650579 | top1:50.4010009765625
31/1445 Data:0.002 | Batch:0.662 | Total:0:00:27 | ETA:0:15:34 | Loss:0.7055240907976704 | top1:49.91511154174805
41/1445 Data:0.002 | Batch:0.664 | Total:0:00:34 | ETA:0:15:34 | Loss:0.7055576647200236 | top1:49.65340042114258
51/1445 Data:0.001 | Batch:0.665 | Total:0:00:41 | ETA:0:15:29 | Loss:0.706027181709514 | top1:49.54592514038086
61/1445 Data:0.001 | Batch:0.669 | Total:0:00:47 | ETA:0:15:20 | Loss:0.7066601475731271 | top1:49.54270935058594
71/1445 Data:0.001 | Batch:0.664 | Total:0:00:54 | ETA:0:15:13 | Loss:0.7072447269735201 | top1:49.42920684814453
81/1445 Data:0.002 | Batch:0.665 | Total:0:01:01 | ETA:0:15

711/1445 Data:0.001 | Batch:0.566 | Total:0:07:56 | ETA:0:05:25 | Loss:0.7036798488238692 | top1:49.97187042236328
721/1445 Data:0.003 | Batch:0.674 | Total:0:08:02 | ETA:0:08:00 | Loss:0.7038223452442397 | top1:49.95838928222656
731/1445 Data:0.001 | Batch:0.677 | Total:0:08:09 | ETA:0:07:56 | Loss:0.7038501189606298 | top1:49.976959228515625
741/1445 Data:0.001 | Batch:0.651 | Total:0:08:16 | ETA:0:07:44 | Loss:0.7038398557185483 | top1:49.990055084228516
751/1445 Data:0.001 | Batch:0.631 | Total:0:08:22 | ETA:0:07:32 | Loss:0.7038212305061351 | top1:49.97897720336914
761/1445 Data:0.003 | Batch:0.616 | Total:0:08:29 | ETA:0:07:39 | Loss:0.7037156652683654 | top1:49.98823928833008
771/1445 Data:0.002 | Batch:0.694 | Total:0:08:36 | ETA:0:07:31 | Loss:0.7036258099762537 | top1:49.99112319946289
781/1445 Data:0.001 | Batch:0.659 | Total:0:08:42 | ETA:0:07:24 | Loss:0.7035241829906322 | top1:50.00135040283203
791/1445 Data:0.001 | Batch:0.668 | Total:0:08:49 | ETA:0:07:14 | Loss:0.70344

1421/1445 Data:0.001 | Batch:0.621 | Total:0:15:47 | ETA:0:00:16 | Loss:0.7002932605065568 | top1:49.93666458129883
1431/1445 Data:0.010 | Batch:0.616 | Total:0:15:54 | ETA:0:00:10 | Loss:0.7002678386903493 | top1:49.91908264160156
1441/1445 Data:0.000 | Batch:0.645 | Total:0:16:00 | ETA:0:00:03 | Loss:0.7002193963751042 | top1:49.92366409301758


/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


161/161 Data:0.002 | Batch:1.149 | Total:0:00:31 | ETA:0:00:00 | Loss:0.6942813460901868 | top1:50.000003814697266

Epoch: [2 | 200] LR: 0.025500
1/1445 Data:1.273 | Batch:2.128 | Total:0:00:02 | ETA:0:51:13 | Loss:0.6995300054550171 | top1:45.78947448730469
11/1445 Data:0.001 | Batch:0.650 | Total:0:00:08 | ETA:0:19:26 | Loss:0.6960196603428234 | top1:50.669857025146484
21/1445 Data:0.002 | Batch:0.693 | Total:0:00:15 | ETA:0:15:35 | Loss:0.6964465635163444 | top1:50.426063537597656
31/1445 Data:0.001 | Batch:0.642 | Total:0:00:21 | ETA:0:15:24 | Loss:0.6956318847594722 | top1:50.88285446166992
41/1445 Data:0.003 | Batch:0.688 | Total:0:00:28 | ETA:0:15:27 | Loss:0.6953556973759721 | top1:51.014122009277344
51/1445 Data:0.010 | Batch:0.664 | Total:0:00:35 | ETA:0:15:34 | Loss:0.6951762192389545 | top1:50.835914611816406
61/1445 Data:0.003 | Batch:0.684 | Total:0:00:41 | ETA:0:15:14 | Loss:0.6952966617756202 | top1:50.82829666137695
71/1445 Data:0.010 | Batch:0.696 | Total:0:00:48 | ET

...

701/1445 Data:0.002 | Batch:0.663 | Total:0:07:43 | ETA:0:08:15 | Loss:0.046790871870824896 | top1:98.21533203125
711/1445 Data:0.002 | Batch:0.664 | Total:0:07:50 | ETA:0:08:08 | Loss:0.046616341303719624 | top1:98.22118377685547
721/1445 Data:0.002 | Batch:0.664 | Total:0:07:56 | ETA:0:08:03 | Loss:0.046585950838342494 | top1:98.22395324707031
731/1445 Data:0.002 | Batch:0.651 | Total:0:08:03 | ETA:0:07:54 | Loss:0.046492236788052095 | top1:98.22449493408203
741/1445 Data:0.002 | Batch:0.664 | Total:0:08:10 | ETA:0:07:47 | Loss:0.046408257648906846 | top1:98.22785949707031
751/1445 Data:0.002 | Batch:0.662 | Total:0:08:16 | ETA:0:07:41 | Loss:0.04643446153907797 | top1:98.2269287109375
761/1445 Data:0.001 | Batch:0.673 | Total:0:08:23 | ETA:0:07:36 | Loss:0.046383542197777274 | top1:98.2267074584961
771/1445 Data:0.002 | Batch:0.666 | Total:0:08:30 | ETA:0:07:30 | Loss:0.046483597788843777 | top1:98.22377014160156
781/1445 Data:0.002 | Batch:0.664 | Total:0:08:36 | ETA:0:07:22 | Loss

1411/1445 Data:0.002 | Batch:0.663 | Total:0:15:32 | ETA:0:00:23 | Loss:0.04647106537859409 | top1:98.24424743652344
1421/1445 Data:0.003 | Batch:0.664 | Total:0:15:38 | ETA:0:00:16 | Loss:0.04646918044525823 | top1:98.24474334716797
1431/1445 Data:0.003 | Batch:0.666 | Total:0:15:45 | ETA:0:00:10 | Loss:0.04646071312938648 | top1:98.244140625
1441/1445 Data:0.000 | Batch:0.664 | Total:0:15:52 | ETA:0:00:03 | Loss:0.04640751527389673 | top1:98.24646759033203
161/161 Data:0.002 | Batch:0.116 | Total:0:00:29 | ETA:0:00:00 | Loss:0.03540547913818304 | top1:98.59764862060547

Epoch: [96 | 200] LR: 0.000749
1/1445 Data:0.595 | Batch:1.262 | Total:0:00:01 | ETA:0:30:26 | Loss:0.05053478106856346 | top1:97.36842346191406
11/1445 Data:0.002 | Batch:0.667 | Total:0:00:07 | ETA:0:17:23 | Loss:0.053384213454344055 | top1:98.08612060546875
21/1445 Data:0.002 | Batch:0.668 | Total:0:00:14 | ETA:0:15:47 | Loss:0.050210932890574135 | top1:98.12030029296875
31/1445 Data:0.001 | Batch:0.667 | Total:0:0

661/1445 Data:0.002 | Batch:0.669 | Total:0:07:04 | ETA:0:08:42 | Loss:0.04734561341616034 | top1:98.21244049072266
671/1445 Data:0.002 | Batch:0.668 | Total:0:07:10 | ETA:0:08:36 | Loss:0.047361296070697986 | top1:98.21475982666016
681/1445 Data:0.002 | Batch:0.667 | Total:0:07:17 | ETA:0:08:30 | Loss:0.047241786580324914 | top1:98.22010803222656
691/1445 Data:0.002 | Batch:0.677 | Total:0:07:24 | ETA:0:08:23 | Loss:0.04722110831479282 | top1:98.22301483154297
701/1445 Data:0.002 | Batch:0.664 | Total:0:07:30 | ETA:0:08:17 | Loss:0.04707319104310641 | top1:98.22959899902344
711/1445 Data:0.002 | Batch:0.665 | Total:0:07:37 | ETA:0:08:09 | Loss:0.046946566421590986 | top1:98.23451232910156
721/1445 Data:0.001 | Batch:0.667 | Total:0:07:44 | ETA:0:08:03 | Loss:0.04680480801238163 | top1:98.23782348632812
731/1445 Data:0.002 | Batch:0.664 | Total:0:07:50 | ETA:0:07:54 | Loss:0.04670294233127695 | top1:98.24393463134766
741/1445 Data:0.002 | Batch:0.665 | Total:0:07:57 | ETA:0:07:49 | Los

1371/1445 Data:0.002 | Batch:0.665 | Total:0:14:51 | ETA:0:00:50 | Loss:0.04672058821261136 | top1:98.24100494384766
1381/1445 Data:0.002 | Batch:0.665 | Total:0:14:58 | ETA:0:00:43 | Loss:0.0468350760594879 | top1:98.23583221435547
1391/1445 Data:0.001 | Batch:0.664 | Total:0:15:04 | ETA:0:00:36 | Loss:0.046863328061239974 | top1:98.23375701904297
1401/1445 Data:0.002 | Batch:0.663 | Total:0:15:11 | ETA:0:00:30 | Loss:0.04688145448789426 | top1:98.23359680175781
1411/1445 Data:0.002 | Batch:0.663 | Total:0:15:18 | ETA:0:00:23 | Loss:0.04683188806650309 | top1:98.23566436767578
1421/1445 Data:0.002 | Batch:0.660 | Total:0:15:24 | ETA:0:00:16 | Loss:0.04679286245480809 | top1:98.23918914794922
1431/1445 Data:0.002 | Batch:0.663 | Total:0:15:31 | ETA:0:00:10 | Loss:0.04677940031206385 | top1:98.24119567871094
1441/1445 Data:0.002 | Batch:0.659 | Total:0:15:38 | ETA:0:00:03 | Loss:0.04675124459502485 | top1:98.24134826660156
161/161 Data:0.002 | Batch:0.120 | Total:0:00:30 | ETA:0:00:00 |

621/1445 Data:0.002 | Batch:0.665 | Total:0:06:49 | ETA:0:09:07 | Loss:0.0454650083054367 | top1:98.2939224243164
631/1445 Data:0.003 | Batch:0.666 | Total:0:06:56 | ETA:0:09:02 | Loss:0.04539467943035267 | top1:98.29844665527344
641/1445 Data:0.001 | Batch:0.661 | Total:0:07:02 | ETA:0:08:54 | Loss:0.045501671329739264 | top1:98.29460144042969
651/1445 Data:0.001 | Batch:0.663 | Total:0:07:09 | ETA:0:08:46 | Loss:0.045365652329103875 | top1:98.29735565185547
661/1445 Data:0.002 | Batch:0.663 | Total:0:07:16 | ETA:0:08:41 | Loss:0.045437576741673355 | top1:98.29047393798828
671/1445 Data:0.002 | Batch:0.663 | Total:0:07:22 | ETA:0:08:34 | Loss:0.04566752104416223 | top1:98.27908325195312
681/1445 Data:0.002 | Batch:0.663 | Total:0:07:29 | ETA:0:08:27 | Loss:0.04557554143523351 | top1:98.28347778320312
691/1445 Data:0.002 | Batch:0.661 | Total:0:07:35 | ETA:0:08:20 | Loss:0.04549754237723786 | top1:98.28471374511719
701/1445 Data:0.001 | Batch:0.661 | Total:0:07:42 | ETA:0:08:14 | Loss: